# Getting started

A lot of people get diagnosed with squamous cell carcinoma in the head & neck region ([HNSCC](https://en.wikipedia.org/wiki/Head_and_neck_cancer)), which frequently metastasizes via the lymphatic system. We set out to develop a methodology to predict the risk of a new patient having metastases in so-called lymph node levels (LNLs), based on their personal diagnose (e.g. findings from a CT scan) and information of previously diagnosed and treated patients. And that's exactly what this code enables you to do as well.

As mentioned, this package is meant to be a relatively simple-to-use frontend. The math is done under the hood and one does not need to worry about it a lot. But let's have a quick look at what we're doing here.

We will assume that you have already read the section on how to install the module and followed its instructions.

## Importing

First, let's import the package.

In [ ]:
import lymph

## Graph

The model is based on the assumption that one can represent the lymphatic system as a directed graph. The arcs in that graph represent the direction of the lymphatic flow and therefore also the direction of metastatic spread. Hence, the first thing to do is to define a graph that represents the drainage pathways of the lymphatic system aptly.

Here, this is done via a dictionary:

In [ ]:
graph = {
    ('tumor', 'primary')  : ['I', 'II', 'III', 'IV'], 
    ('lnl'  , 'I')  :       ['II'], 
    ('lnl'  , 'II') :       ['III'], 
    ('lnl'  , 'III'):       ['IV'], 
    ('lnl'  , 'IV') :       []
}

Every key in this dictionary is a tuple of the form `({type}, {name})` and represents either a tumor - in which case the `{type}` must be `'tumor'` - or a lymph node level (`{type}` must be `'lnl'`). The value of each of those nodes is then a list of names for nodes it connects to. So, for example the primary tumor `('tumor', 'primary')` in the `graph` above has directed arcs to `a` and `b`, while the LNL `c` does not have any outgoing connections.

We can simply create an instance of `System` using only this graph and let it report itself:

In [ ]:
unilateral_model = lymph.Unilateral(graph=graph)
print(unilateral_model)

The percentages between two nodes represents the probability rate that metastatic spread occurs along it. In the case of the tumor spreading to LNL `a` we call this probability *base probability rate* $\tilde{b}_a$. For the spread between lymph node levels, we call it *transition probability rate*, e.g. $\tilde{t}_{ab}$. The difference to the base probability rate is that it only plays a role if the parent LNL is already ivolved with metastases, while the tumor always spreads, of course.

We can change these spread probability rates by setting the attribute `spread_probs` of the `System`. It can be set with an array of these spread sprobability rates.

In [ ]:
unilateral_model.spread_probs = [0.05, 0.3, 0.2, 0.15, 0.1, 0.25, 0.2]
print(unilateral_model)

Reversely, we can also read them out:

In [ ]:
spread_probabilities = unilateral_model.spread_probs
print(spread_probabilities)

## Diagnostic Modalities

To ultimately compute the likelihoods of observations, we need to fix the sensitivities and specificities of the obtained diagnoses. And since we might have multiple diagnostic modalities available, we need to tell the system which of them comes with which specificity and sensitivity. We do this by creating a dictionary of specificity/sensitivity pairs:

In [ ]:
mri_and_pet_spsn = {"MRI": [0.63, 0.81], 
                    "PET": [0.86, 0.79]}
#                           ^     ^
#                  specificty     sensitivity

Now we can pass this to the system by setting the `modalities` attribute, which expects a dictionary containing the diagnostic modalities and as a key to it the numbers for specificity & sensitivity.

In [ ]:
unilateral_model.modalities = mri_and_pet_spsn

## Data / Observations

To compute the likelihood of a set of probability (rates) given a patient cohort we need such a patient cohort, of course. We can provide it to the system in the form of a `pandas` `DataFrame`. Here is an example:

In [ ]:
import pandas as pd
from lyscripts.data.clean import lyprox_to_lymph

dataset_url = "https://raw.githubusercontent.com/rmnldwg/lydata/main/2021-usz-oropharynx/data.csv"
example_cols = [
    ("info", "t_stage"),
    ("PET", "I"),
    ("PET", "II"),
    ("PET", "III"),
    ("PET", "IV"),
    ("MRI", "I"),
    ("MRI", "II"),
    ("MRI", "III"),
    ("MRI", "IV"),
]

dataset = lyprox_to_lymph(pd.read_csv(dataset_url, header=[0,1,2]))
dataset[example_cols]

Note that this data has two header-rows, defining not only the individual column's content, but also to which over-arching category they belong. The "Info" category plays a special role here along with its sub-category "T-stage". It will later tell the system which time prior to use according to a dictionary of these distributions.

The "pathology" section denotes that this dataset contains observations from a pathologic diagnostic modality (neck dissections in this case). How this is termed is irrelevant, as we will be telling the system what to look for. Import is, however, that - if we had multiple diagnostic modalities - they all contain a column for each lymph node level in the system we have set up. Obvioulsy, this dataset here does not match the system set up earlier, so let's fix that.

In [ ]:
unilateral_model.modalities = {"PET": [0.86, 0.79]}

To feed the dataset into the system, we assign the dataset to the attribute `patient_data`. What the system then does here is creating a diagnose matrix for every T-stage in the data.

In [ ]:
unilateral_model.patient_data = dataset

As you can see, we get a warning that we have no distributions defined for marginalizing over diagnose times. This is the next step.

## Distribution over diagnose times

The last ingredient to set up (at least when using the hidden Markov model) would now be the time prior. Since this dataset contains only early T-stage patients the exact shape does not matter too much, as long as it is "reasonable". If we also had late T-stage patients in the cohort, we would need to think about how the two time priors relate to each other.

For now we are going to use binomial distributions for this. Their shape makes intuitive sense: Since the time prior $p_T(t)$ is a distribution over the probability of diagnosing a patient after $t$ time steps, given his T-stage $T$ we would expect that a very early detection of the cancer is similarly unlikely as a very late one.

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

max_t = 10
time_steps = np.arange(max_t+1)
p = 0.4

early_prior = sp.stats.binom.pmf(time_steps, max_t, p)

plt.plot(time_steps, early_prior, "o-");
plt.xlabel("time step $t$");
plt.ylabel("probability $p$");

In [ ]:
unilateral_model.diag_time_dists["early"] = early_prior

In [ ]:
from scipy.special import factorial

def binom_pmf(k: np.ndarray, n: int, p: float):
    """Binomial PMF"""
    if p > 1. or p < 0.:
        raise ValueError("Binomial prob must be btw. 0 and 1")
    q = (1. - p)
    binom_coeff = factorial(n) / (factorial(k) * factorial(n - k))
    return binom_coeff * p**k * q**(n - k)

def parametric_binom_pmf(n: int):
    """Return a parametric binomial PMF"""
    def inner(t, p):
        """Parametric binomial PMF"""
        return binom_pmf(t, n, p)
    return inner

In [ ]:
unilateral_model.diag_time_dists["late"] = parametric_binom_pmf(max_t)

## Likelihood

With everything set up like this, we can compute the likelihood of seeing the above dataset given a set of base and transition probability (rates).

In [ ]:
test_probabilities = np.array([0.02, 0.24, 0.03, 0.2, 0.23, 0.18, 0.18, 0.5])

llh = unilateral_model.likelihood(given_params=test_probabilities, log=True)

print(f"log-likelihood is {llh:.2f}")

From here it is up to the user what to do with this quantity. Most *likely* though, one would want to perform MCMC sampling with this.

## Summary

To set up a model for lymphatic metastatic spread, you need to do the following things:

1. Define a graph that connects the lymph node levels via a dictionary
2. Provide the specificity & sensitivity of the diagnostic modalities to the `modalities` attribute
3. Assign your correctly formatted pandas `DataFrame` to the attribute `patient_data` of the model
4. For each T-stage in the data, define a distribution over possible diagnose times

Then, you can use the `log_likelihood` method to compute the log-likelihood of the stored data given an array of parameters - the spread probabilities.